In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q accelerate
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [ ]:
!pip install -U langchain-openai

In [1]:
# Document Loader and text splitter imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
# LLM Model import:
from langchain.chat_models import ChatOpenAI
# Langchain Pipeline and Template import
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

In [3]:
import json

# openAI key 
with open('preset.json') as f:
    data = json.load(f)

# model_name = "gpt-4-0125-preview"
model_name = "gpt-4"
max_token_num = 8192
llm = ChatOpenAI(temperature=0, openai_api_key=data["OPENAI_API_KEY"], model_name=model_name)

In [4]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:    
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [82]:
# import text from txt
with open('EuroParl/Task_A/11-07-07-003/transcript.txt', 'r') as file:
    text = file.read()

print("Num of Tokens:",num_tokens_from_string(text, model_name))

splitter = RecursiveCharacterTextSplitter()
from langchain.docstore.document import Document
docs = splitter.split_text(text)
docs = [Document(page_content=t) for t in docs]

Num of Tokens: 9662


In [5]:
text = "Grazie, signor Presidente. Chiedo questo perché si tratta di una proposta di ordinanza che approva il piano del quartiere di Globeville. E vorrei chiedere una cortesia di audizione pubblica per la notte del 1° dicembre. 1° dicembre, ricevuto. Quindi, se non ci sono obiezioni da parte dei membri del consiglio, avremo una breve audizione pubblica sul 971, il Piano del Quartiere di Globeville. Lunedì 1° dicembre. Grazie, Consigliere Monti. Signora Segretaria, può proporre il prossimo punto, che credo dovrebbe essere il 924, chiamato dal Consigliere Fox? Consigliere Fox, cosa vorrebbe fare con questo? Potremmo metterlo in votazione? Certamente. Consigliere S, potrebbe presentare la mozione questa sera? Sì, signor Presidente. Potrebbe mettere in discussione il 924 per la considerazione finale e per il passaggio? Certamente, propongo che il 924 sia posto per la considerazione finale e per il passaggio. È stato presentato e sostenuto. Commenti da parte dei membri del Consiglio, Consigliera Fox. Grazie, signor Presidente. Questa è un'ordinanza per l'acquisto di un edificio per scopi di emergenza 911 che ospiterà i ricevitori delle chiamate e le informazioni di dispacciamento. Non sono contrario all'acquisto effettivo di questo edificio, ma è un pacchetto completo. Viene anche con una fattura che per il pagamento richiede la fluttuazione di certificati di partecipazione e infatti"

print("Num of Tokens:",num_tokens_from_string(text, model_name))

splitter = RecursiveCharacterTextSplitter()
from langchain.docstore.document import Document
docs = splitter.split_text(text)
docs = [Document(page_content=t) for t in docs]

Num of Tokens: 393


In [6]:
# https://github.com/ELITR/automin-2021 DATA

prompt_template = """
Sei un assistente AI specializzato nel riassumere meeting in maniera precisa.
Utilizza le seguenti regole:
- Utilizza Bullet Points per riassumere il testo
- Prima di ogni bullet point metti il nome della persona che ha parlato
- Cita i partecipanti all'inizio del riassunto 

{text}

RIASSUNTO IN ITALIANO CON BULLET POINTS:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

In [86]:
# GPT-3.5-Turbo
print(chain.run(docs))

Partecipanti: PERSON27, PERSON54, PERSON55

- PERSON27 chiede informazioni su come funziona il link per le riunioni su Zoom
- PERSON54 spiega che il link per le riunioni su Zoom è lo stesso ogni volta
- PERSON27 conferma che il link funzionerà anche per le prossime riunioni
- PERSON54 propone di includere il link Zoom e il link Google nelle future riunioni
- PERSON27 discute sull'importanza delle annotazioni manuali e dei minuti originali delle riunioni
- PERSON54 chiede come vengono corrette le annotazioni e i minuti
- PERSON27 spiega il processo di correzione delle annotazioni e dei minuti
- PERSON54 chiede se includere l'allineamento basato sull'annotazione nelle valutazioni
- PERSON27 e PERSON54 discutono sull'utilizzo di misure automatiche o valutazioni umane per le valutazioni
- PERSON27 propone di utilizzare la valutazione umana se l'allineamento non è chiaro
- PERSON55 suggerisce di discutere con PERSON7 prima di prendere decisioni sulle valutazioni
- PERSON27 e PERSON55 discut

In [79]:
# gpt-4-0125-preview
print(chain.run(docs))

Partecipanti: PERSON27, PERSON54, PERSON55

- **PERSON27**: Esprime soddisfazione per la possibilità di registrare e chiede chiarimenti sull'uso del link di Zoom per le riunioni future.
- **PERSON54**: Conferma che il link di Zoom rimane lo stesso per tutte le riunioni e discute l'uso di Google Meet e Zoom, menzionando che Google Meet diventerà a pagamento per sessioni più lunghe di un'ora.
- **PERSON27**: Chiede e riceve conferma che il documento "Minuting annotation guidelines" è disponibile e discute l'output del processo di annotazione, che include trascrizioni e verbali corretti manualmente.
- **PERSON54**: Solleva dubbi sull'efficacia delle misure automatiche per la valutazione delle sommarizzazioni e suggerisce l'uso di valutazioni umane come complemento.
- **PERSON55**: Interviene per chiarire il ruolo dell'annotazione di allineamento e la necessità di una misura di valutazione manuale, sottolineando l'importanza di una valutazione umana accurata.
- **PERSON27**: Riassume la di

In [7]:
# gpt-4-0125-preview
print(chain.run(docs))

c:\Users\franc\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Partecipanti: Signor Presidente, Consigliere Monti, Signora Segretaria, Consigliere Fox, Consigliere S.

- Signor Presidente: Ha ricevuto la richiesta di una proposta di ordinanza per approvare il piano del quartiere di Globeville. Ha fissato una audizione pubblica per la notte del 1° dicembre.
- Consigliere Monti: Ha richiesto un'audizione pubblica per la notte del 1° dicembre riguardo al Piano del Quartiere di Globeville.
- Signora Segretaria: Ha proposto il prossimo punto, il 924, chiamato dal Consigliere Fox.
- Consigliere Fox: Ha chiesto di mettere in votazione il punto 924.
- Consigliere S: Ha presentato la mozione per mettere in discussione il punto 924 per la considerazione finale e per il passaggio.
- Consigliere Fox: Ha commentato sull'ordinanza per l'acquisto di un edificio per scopi di emergenza 911, esprimendo non essere contrario all'acquisto dell'edificio, ma ha sollevato preoccupazioni riguardo alla fattura associata che richiede la fluttuazione di certificati di partec